In [1]:
# read credentials from file 
import pandas as pd
import json

credentials = pd.read_json('credentials.json')

In [2]:
# establish ssh connection to the host 
import paramiko 
host = "artemis.uni.ma"
port = 22
username = credentials['user'][0]
password = credentials['pwd'][0]

ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect(host, port, username, password)

# Complete Process
continuously monitor the bestblockhash - returns the tip of the blockchain

In [3]:
def getBestBlockHash():
    # get best blockhash - returns the header hash of the most recent block on the best block chain
    stdin, stdout, stderr = ssh.exec_command("bitcoin-cli getblockchaininfo")
    blockchaininfo = json.loads(stdout.read().decode("utf-8"))
    bestblockhash = blockchaininfo['bestblockhash']
    return bestblockhash

In [4]:
def getBlockHash(height):
    # retrieve block hash by height of block
    stdin, stdout, stderr = ssh.exec_command(f"bitcoin-cli getblockhash {height}")
    block_hash = stdout.read().decode("utf-8")[:-1]
    return block_hash


In [5]:
def getblock(blockhash): 
    command = "bitcoin-cli getblock " + blockhash + " 2"
    stdin, stdout, stderr = ssh.exec_command(command)
    block = json.loads(stdout.read().decode("utf-8"))

    # filter relevant blockinfo
    from datetime import datetime, timezone
    ts_epoch = block['time']
    block_timestamp = datetime.fromtimestamp(ts_epoch)
    block_median_time = block_timestamp.strftime('%Y-%m-%dT%H:%M')
    block_date = block_timestamp.strftime('%Y-%m-%d')
    block_hash = block['hash']
    block_height = block['height']
    # handling genesis block
    if block_height != 0:
        previousblockhash = block['previousblockhash']

    # build json object
    data = {}
    data['block_hash'] = block_hash
    data['block_height'] = block_height
    data['block_median_time'] = str(block_median_time) # change timestamp to median_time
    data['block_date'] = str(block_date)
    if block_height != 0:
        data['previousblockhash'] = previousblockhash
        

    return data, block

In [6]:
#example block information retrieval
data, block = getblock(getBlockHash(600000))
json_data = json.dumps(data, indent=4, sort_keys=False)
#print(json_data)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

# Transactions

In [52]:
from datetime import datetime, timezone

def gettx(tx):
        
    command = "bitcoin-cli getrawtransaction " + tx['txid'] + " true"
    stdin, stdout, stderr = ssh.exec_command(command)

    # load data into json object rawtx
    rawtx = json.loads(stdout.read().decode("utf-8"))

    txdata = {}
    txdata['txid'] = tx['txid']
    txdata['block_hash'] = block['hash']
    ts_epoch = block['time']
    block_timestamp = datetime.fromtimestamp(ts_epoch)
    block_date = block_timestamp.strftime('%Y-%m-%d')
    txdata['block_date'] = str(block_date)


    addr = ""
    val = 0
    outSum = 0
    inSum = 0
    inputAddrObject = {}
    outputAddrObject = {}
    input_address_list = [] 
    output_address_list = []

    ## check coinbase 
    if 'coinbase' not in rawtx['vin'][0]: 
        for i in rawtx['vin']:  
            command = "bitcoin-cli getrawtransaction " + i['txid'] + " true"
            stdin, stdout, stderr = ssh.exec_command(command)
            inputtx = json.loads(stdout.read().decode("utf-8"))
            inputtx_json_data = json.dumps(inputtx, indent=4, sort_keys=False)
            val = int (inputtx['vout'][i['vout']]['value']*100000000)
            inSum += val 

            # input addresses
            for ia in inputtx['vout'][i['vout']]['scriptPubKey']['addresses']: 
                inputAddrObject['addr'] = ia
                inputAddrObject['val'] = val

                jInAddr = json.dumps(inputAddrObject)
                jsonInDict = json.loads(jInAddr)
                input_address_list.append(jsonInDict)

        # output addresses
        for o in rawtx['vout']:
            if o['scriptPubKey']['type'] != "nulldata": # handling OP_RETURN data - can be skipped
                outSum += o['value']
                for a in o['scriptPubKey']['addresses']:
                    outputAddrObject['addr'] = a
                    outputAddrObject['outNr'] = o['n']
                    outputAddrObject['val'] = o['value']

                    jOutAddr = json.dumps(outputAddrObject)
                    jsonOutDict = json.loads(jOutAddr)
                    output_address_list.append(jsonOutDict)
    else: 
        for o in rawtx['vout']:   
            if o['scriptPubKey']['type'] != "nulldata": # handling OP_RETURN data - can be skipped
                outSum += o['value']
                for a in o['scriptPubKey']['addresses']:
                    outputAddrObject['addr'] = a
                    outputAddrObject['outNr'] = o['n']

        jOutAddr = json.dumps(outputAddrObject)
        jsonOutDict = json.loads(jOutAddr)
        output_address_list.append(jsonOutDict)
        inSum = int(outSum*100000000)
        inputAddrObject['addr'] = "coinbase"
        inputAddrObject['val'] = inSum

        jInAddr = json.dumps(inputAddrObject)
        jsonInDict = json.loads(jInAddr)
        input_address_list.append(jsonInDict)


    # get degrees
    txdata['outDegree'] = len(rawtx['vout'])
    txdata['inDegree'] = len(rawtx['vin'])



    x = round(outSum*100000000, 0)
    txdata['outSum'] = int( x)
    txdata['inSum'] = inSum
    txdata['input_list'] = input_address_list  
    txdata['output_list'] = output_address_list 
        
        
    return txdata

In [53]:
tx = block['tx'][1]
txlist = gettx(tx)
txlist

{'txid': 'a8178a7223372414ac060b4bba4b33b8b4847a756fa76a715af7fd11bfd143d5',
 'block_hash': '00000000000000000007316856900e76b4f7a9139cfbfba89842c8d196cd5f91',
 'block_date': '2019-10-19',
 'outDegree': 3,
 'inDegree': 1,
 'outSum': 138819784059,
 'inSum': 138819884059,
 'input_list': [{'addr': '17A16QmavnUfCW11DAApiJxp7ARnxN5pGX',
   'val': 138819884059}],
 'output_list': [{'addr': '3QKAn2B1uDquujLZnoynVoq1M9uac66Ysr',
   'outNr': 0,
   'val': 0.00795759},
  {'addr': '1F8fDpYbMLMaz1tBEehqPJSN8XTL6t5TDz',
   'outNr': 1,
   'val': 0.01241006},
  {'addr': '17A16QmavnUfCW11DAApiJxp7ARnxN5pGX',
   'outNr': 2,
   'val': 1388.17747294}]}

In [46]:
command = "bitcoin-cli getrawtransaction " + tx['txid'] + " true"
stdin, stdout, stderr = ssh.exec_command(command)

# load data into json object rawtx
rawtx = json.loads(stdout.read().decode("utf-8"))
'coinbase' in rawtx['vin'][0]#['coinbase'] 

False

In [47]:
tx

{'txid': 'a8178a7223372414ac060b4bba4b33b8b4847a756fa76a715af7fd11bfd143d5',
 'hash': 'a8178a7223372414ac060b4bba4b33b8b4847a756fa76a715af7fd11bfd143d5',
 'version': 2,
 'size': 290,
 'vsize': 290,
 'weight': 1160,
 'locktime': 0,
 'vin': [{'txid': '886781640b20eb9f93e9d068eda3d676ea48c7a0c5aebc97d06c058fcebfe92e',
   'vout': 4,
   'scriptSig': {'asm': '3045022100a30ad9317573aa3d6e9c668bc510e6a828261bc2db08f2fb03de901a3fb9370e02204396fd8c14ab2e0355981c7a88da103f874e47d49bf3148180b061568a566907[ALL] 047146f0e0fcb3139947cf0beb870fe251930ca10d4545793d31033e801b5219abf56c11a3cf3406ca590e4c14b0dab749d20862b3adc4709153c280c2a78be10c',
    'hex': '483045022100a30ad9317573aa3d6e9c668bc510e6a828261bc2db08f2fb03de901a3fb9370e02204396fd8c14ab2e0355981c7a88da103f874e47d49bf3148180b061568a5669070141047146f0e0fcb3139947cf0beb870fe251930ca10d4545793d31033e801b5219abf56c11a3cf3406ca590e4c14b0dab749d20862b3adc4709153c280c2a78be10c'},
   'sequence': 4294967295}],
 'vout': [{'value': 0.00795759,
   'n': 